In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

In [ ]:
class DiceBCEWithLogisticLoss(nn.Module):
  def __init__(self, weight=None, reduction='mean', BCE_weight=0.4):
    super(DiceBCEWithLogisticLoss, self).__init__()
    self.reduction = reduction
    self.BCE_weight = BCE_weight

  def forward(self, inputs, targets):
    logits = inputs  
    inputs = torch.sigmoid(inputs) 

    assert inputs.size() == targets.size()

    # inputs = inputs.view(-1)
    # targets = targets.view(-1)
    
    intersection = (inputs * targets).sum()
    union = inputs.sum() + targets.sum()               
    
    dice_loss = 1 - ((2.*intersection) / (union))  
    BCE_loss = F.binary_cross_entropy_with_logits(
        input=logits,
        target=targets,
        reduction=self.reduction
    )
    Dice_BCE = BCE_loss * self.BCE_weight + dice_loss
    return Dice_BCE

In [ ]:
class CustomAccuracyLoss(nn.Module):
  def __init__(self, tn_weight=0.2):
    super(CustomAccuracyLoss, self).__init__()
    self.tn_weight = tn_weight

  def forward(self, inputs, targets):  
    inputs = torch.sigmoid(inputs) 

    assert inputs.size() == targets.size()

    # inputs = inputs.view(-1)
    # targets = targets.view(-1)
    
    tp = (inputs * targets).sum()
    tn = ((1 - inputs).abs() * (1 - targets).abs()).sum()
    fp = (inputs * (1 - targets).abs()).sum()
    fn = ((1 - inputs).abs() * targets).sum()

    tn_w = tn*self.tn_weight

    accuracy = (tp + tn_w)/(tp + tn + fp + fn)

    ratio = (tn + fp)/(tp + fn)
    max = (1 + self.tn_weight*ratio)/(1 + ratio)

    accuracy_loss = 1 - accuracy/max
    return accuracy_loss

In [ ]:
class FocalTverskyLoss(nn.Module):
  def __init__(self, alpha=0.7, gamma=0.75, smooth=1.):
    super(FocalTverskyLoss, self).__init__()
    self.alpha = alpha
    self.beta = 1 - self.alpha
    self.gamma = gamma
    self.smooth = smooth

  def forward(self, inputs, targets):  
    inputs = torch.sigmoid(inputs) 

    assert inputs.size() == targets.size()

    # inputs = inputs.view(-1)
    # targets = targets.view(-1)
    
    tp = (inputs * targets).sum()
    tn = ((1 - inputs).abs() * (1 - targets).abs()).sum()
    fp = (inputs * (1 - targets).abs()).sum()
    fn = ((1 - inputs).abs() * targets).sum()
    
    tversky = (tp + self.smooth) / (tp + self.alpha*fn + self.beta*fp + self.smooth)
    tversky_loss = 1 - tversky

    focal_tversky = torch.pow(tversky_loss, self.gamma)
    return focal_tversky

In [ ]:
def train_incorporate(
    loader, 
    model, 
    optimizer_class, 
    optimizer_mask, 
    loss_fn_class, 
    loss_fn_mask, 
    scaler, 
    device, 
    # weight_class=1, 
    # weight_mask=1
  ):
  # batch_size = loader.batch_size
  # dataset_size = len(loader.dataset)
  total_train_loss_class = total_train_loss_mask = 0
  tp = tn = fp = fn = 0
  num_correct = num_pixels = 0
  dice_score = count_dice = 0
  
  loop = tqdm(loader)
  model.train()

  for batch_index, (image, label, mask) in enumerate(loop):
    data = image.to(device)

    label = label.long().to(device)
    filter = label.reshape(-1, 1)
    target_mask = mask.float().unsqueeze(1).to(device)

    optimizer_mask.zero_grad()
    optimizer_class.zero_grad()

    with torch.cuda.amp.autocast():
      output_class, output_mask = model(data)
      target_class = torch.zeros_like(output_class, device=device)
      target_class[np.arange(data.size(0)), label] = 1
      loss_mask = loss_fn_mask(output_mask[filter==1], target_mask[filter==1])
      # loss_mask = loss_fn_mask(output_mask, target_mask)
      loss_class = loss_fn_class(output_class, target_class)
      # loss = loss_class * weight_class + loss_mask * weight_mask

    scaler.scale(loss_mask).backward(retain_graph=True)
    scaler.scale(loss_class).backward()

    scaler.step(optimizer_mask)
    scaler.step(optimizer_class)

    scaler.update()

    with torch.no_grad():
      prediction_class = output_class.argmax(dim=1, keepdim=True)
      # accuracy_class = prediction_class.eq(label.view_as(prediction_class)).sum()

      truth_class = label.view_as(prediction_class)
      tp += (prediction_class * truth_class).sum()
      tn += ((1 - prediction_class).abs() * (1 - truth_class).abs()).sum()
      fp += (prediction_class * (1 - truth_class).abs()).sum()
      fn += ((1 - prediction_class).abs() * truth_class).sum()

      prediction_mask = torch.sigmoid(output_mask)
      prediction_mask = (prediction_mask > 0.5).float()

      # #### save for investigation
      # torchvision.utils.save_image(prediction_mask, f"test/preds/pred_{batch_index}.jpg")
      # torchvision.utils.save_image(target_mask, f"test/labels/label_{batch_index}.jpg")

      num_correct += (prediction_mask[truth_class==1] == target_mask[truth_class==1]).sum()
      num_pixels += torch.numel(prediction_mask[truth_class==1])
      if torch.sum(truth_class==1) > 0:
        dice_score += (2 * (prediction_mask[truth_class==1] * target_mask[truth_class==1]).sum()) / ((prediction_mask[truth_class==1] + target_mask[truth_class==1]).sum())
        # count_dice += torch.sum(truth_class==1)
        count_dice += 1

    loss_item_class = loss_class.item()
    loss_item_mask = loss_mask.item()

    loss = loss_item_class + loss_item_mask

    loop.set_postfix(loss_class=loss_item_class, loss_mask=loss_item_mask)  

    total_train_loss_class += loss_item_class
    total_train_loss_mask += loss_item_mask

  percision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * percision * recall / (percision + recall) 
  accuracy_class = (tp + tn) / (tp + tn + fp + fn) * 100
  dice_class = tp / (fp + fn + tp) * 100

  accuracy_mask = num_correct / num_pixels * 100
  dice_mask = dice_score / count_dice * 100

  print("\n[TRAIN]")
  print(f"[Classification]: Loss: {total_train_loss_class/len(loader):.6f}, Dice: {dice_class:2f}, Acc: {accuracy_class:2f}, F1: {f1:2f},")
  print(f"[Segmentation]:   Loss: {total_train_loss_mask/len(loader):.6f}, Dice: {dice_mask:2f},  Acc: {accuracy_mask:2f}")
  print(f"[Confusion]:      TP: {tp.item()}, TN: {tn.item()}, FP: {fp.item()}, FN: {fn.item()}")

  return total_train_loss_class/len(loader), total_train_loss_mask/len(loader), accuracy_class, accuracy_mask, dice_class, dice_mask

In [ ]:
def train_single(loader, model, optimizer, loss_fn, scaler, device):
  total_train_loss = 0
  tp = tn = fp = fn = 0
  
  loop = tqdm(loader)
  model.train()

  for batch_index, (image, label, mask) in enumerate(loop):
    data = image.to(device)
    label = label.long().to(device)

    with torch.cuda.amp.autocast():
      output_class = model(data)
      target_class = torch.zeros_like(output_class, device=device)
      target_class[np.arange(data.size(0)), label] = 1
      loss_class = loss_fn(output_class, target_class)
      loss = loss_class

    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    with torch.no_grad():
      prediction_class = output_class.argmax(dim=1, keepdim=True)

      truth_class = label.view_as(prediction_class)
      tp += (prediction_class * truth_class).sum()
      tn += ((1 - prediction_class).abs() * (1 - truth_class).abs()).sum()
      fp += (prediction_class * (1 - truth_class).abs()).sum()
      fn += ((1 - prediction_class).abs() * truth_class).sum()

    loop.set_postfix(loss=loss.item())  
    total_train_loss += loss.item() 

  percision = tp / (tp + fp)
  recall = tp / (tp + fn)

  f1 = 2 * percision * recall / (percision + recall) 
  accuracy_class = (tp + tn) / (tp + tn + fp + fn) * 100
  dice_class = tp / (fp + fn + tp) * 100

  print("\n[TRAIN]:         Training Loss: {:.6f}".format(total_train_loss / len(loader)))
  print(f"[Classification]: Dice: {dice_class:2f}, Acc: {accuracy_class:2f}, F1: {f1:2f},")
  print(f"[Confusion]:      TP: {tp.item()}, TN: {tn.item()}, FP: {fp.item()}, FN: {fn.item()}")

  loss_mask = accuracy_mask = dice_mask = 0
  
  return total_train_loss/len(loader), loss_mask, accuracy_class, accuracy_mask, dice_class, dice_mask